# Train the network using the REDISAI db as an exchange place and debug the problems


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.models as models
import torch.utils.data as tdata

import redisai as rai

from dataclasses import dataclass

# import the modules used in the program
import train_utils

c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


## Create the network

In [2]:
@dataclass
class TrainParams:
    ps_id: str
    N: int
    task: str
    func_id: int
    lr: float
    batch_size: int
    

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    
def create_model(init: bool):
    """Creates the model used to train the network

    For this example we'll be using the simple model from the MNIST examples
    (https://github.com/pytorch/examples/blob/master/mnist/main.py)
    """

    def init_weights(m: nn.Module):
        """Initialize the weights of the network"""
        if isinstance(m, nn.Conv2d):
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0.01)
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0.01)

    # Create the model and initialize the weights
    model = Net()

    # If the task is initializing the layers do so
    if init:
        print('Initializing layers...')
        model.apply(init_weights)

    return model

In [3]:
torch.cuda.is_available()

True

## Create the Dataloader

In [4]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

train_data = datasets.MNIST('./data', train=True, download=False, transform=transform)
val_data = datasets.MNIST('./data', train=False, download=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=128)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=128)

In [5]:
len(train_loader), len(val_loader)

(469, 79)

# Define the train and test methods


In [37]:
def train(model: nn.Module, device,
          train_loader: tdata.DataLoader,
          optimizer: torch.optim.Optimizer, tensor_dict) -> float:
    """Loop used to train the network"""
    model.train()
    loss, tot = 0, 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        loss = F.nll_loss(output, target)
        tot += loss.item()
        loss.backward()

        # Here save the gradients to publish on the database
        train_utils.update_tensor_dict(model, tensor_dict)
        optimizer.step()
        

        if batch_idx % 15 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                1, batch_idx * len(data), len(train_loader.dataset),
                   100. * batch_idx / len(train_loader), loss.item()))

    return tot/len(train_loader)


def validate(model, device, val_loader: tdata.DataLoader) -> (float, float):
    """Loop used to validate the network"""
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(val_loader.dataset)

    accuracy = 100. * correct / len(val_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    return accuracy, test_loss

## Main entrypoint of the code

In [38]:
from copy import deepcopy

params = TrainParams(ps_id='example', func_id=0, N =2, task='train', lr=0.01, batch_size=128)


torch.manual_seed(42)
device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create the model

model = create_model(True).to(device)
model

initial = deepcopy(model.state_dict())

train_utils.save_model_weights(model, params, con)

Initializing layers...


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
initial['conv2.bias']
torch.set_printoptions(precision=10)

In [12]:
# Create the redis connection
addr = '192.168.99.101'
port = 31618
con = rai.Client(debug=True, host=addr, port=port)

### Train for a couple of epochs

In [39]:
# create the tensor dict
tdict = dict()

optimizer = optim.SGD(model.parameters(), lr=0.01)
train(model, device, train_loader, optimizer, tdict)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.331917
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.153557
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.857306
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.489405
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.185412
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.781897
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.010033
Train Epoch: 1 [13440/60000 (22%)]	Loss: 0.578734
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.691369
Train Epoch: 1 [17280/60000 (29%)]	Loss: 0.583061
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.599189
Train Epoch: 1 [21120/60000 (35%)]	Loss: 0.498211
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.755305
Train Epoch: 1 [24960/60000 (42%)]	Loss: 0.482500
Train Epoch: 1 [26880/60000 (45%)]	Loss: 0.550762
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.522056
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.474251
Train Epoch: 1 [32640/60000 (54%)]	Loss: 0.604446
Train Epoch: 1 [34560/60000 (58%)]	Loss: 0.461607
Train Epoch: 1 [36480/60000 (61%)]	Loss: 0.394812
Train Epoch:

0.7277375650939657

In [40]:
[(k, torch.mean(v)) for k, v in tdict.items()]

[('conv1.weight.grad', tensor(0.0010779696, device='cuda:0')),
 ('conv1.bias.grad', tensor(0.0054637371, device='cuda:0')),
 ('conv2.weight.grad', tensor(0.0006221278, device='cuda:0')),
 ('conv2.bias.grad', tensor(0.0024095972, device='cuda:0')),
 ('fc1.weight.grad', tensor(0.0002331834, device='cuda:0')),
 ('fc1.bias.grad', tensor(0.0006510335, device='cuda:0')),
 ('fc2.weight.grad', tensor(-1.3606040739e-09, device='cuda:0')),
 ('fc2.bias.grad', tensor(-3.7252902985e-09, device='cuda:0'))]

In [19]:


train_utils.save_gradients(tdict,params, con)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
model.conv2.bias

In [ ]:
tdict['conv2.bias.grad']

In [ ]:
torch.eq(model.conv2.weight, initial['conv2.weight']).all()

In [ ]:
import numpy as np
a = con.tensorget("example:conv2.bias.grad/0", as_numpy=False)
a